<a href="https://colab.research.google.com/github/ugohenrique/Ifsp/blob/master/tarefa8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
importar as bibliotecas necessárias
"""
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms
#definir o tamanho do lote
batch_size = 50
# Dados de Treinamento
train_data_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229,
            0.224, 0.225])])
#Conjunto de treino com dados do CIFAR10
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
       transform=train_data_transform)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size, shuffle=True, num_workers=2)

#ajustar os dados visto que CIFAR é 28x28 e imagenet 224x224
"""
 É necessário padronizar os dados do CIFAR-10 utilizando a média e o desvio-padrão da ImageNet,
 e é necessário adicionar um pequeno aumento de dados (flip):
"""
val_data_transform = transforms.Compose([transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229,0.224, 0.225]) ])
val_set = torchvision.datasets.CIFAR10(root='./data',train=False, download=True,
       transform=val_data_transform)
val_order = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)

#Escolher um dispositivo, preferencialmente GPU:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Definir o modelo de treinamento usando PyTorch.
def train_model(model, loss_function, optimizer,data_loader):
    # Definindo o modelo para o modo de treinamento
    model.train()
    #variavel para armazenar a perda
    current_loss = 0.0
    #variavel para guardar a precisão
    current_acc = 0
    # Repete sobre o conjunto de treinamento
    for i, (inputs, labels) in enumerate(data_loader):
        # Envia a entrada ou os rótulos para a GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        # reinicia os parâmetros do gradiente
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            # Passo adiante
            outputs = model(inputs) 
            _, predictions = torch.max(outputs, 1)
            loss = loss_function(outputs, labels)
            # passo para trás
            loss.backward()
            optimizer.step()
        # Estatística
        current_loss += loss.item() * inputs.size(0)
        current_acc += torch.sum(predictions == labels.data)
        total_loss = current_loss / len(data_loader.dataset)
        total_acc = current_acc.double() / len(data_loader.dataset)
        print('Train Loss: {:.4f}; Accuracy:{:.4f}'.format(total_loss, total_acc))

#Definir o primeiro cenário de transferência de aprendizado
def tl_feature_extractor(epochs=3):
    # Carregar o modelo pré-treinado
    model = torchvision.models.resnet18(pretrained=True)
    # Excluir os parâmetros existentes do passo para trás
    for param in model.parameters():
        param.requires_grad = False
        # As novas camadas require, por padrão,
        # requires_grad=True
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)
        # Transferindo para a GPU (se disponível)
        model = model.to(device)
        loss_function = nn.CrossEntropyLoss()
        # Otimizar apenas os parâmetros da última camada
        optimizer = optim.Adam(model.fc.parameters())
        # treinamento
        for epoch in range(epochs):
            print('Epoch {}/{}'.format(epoch + 1, epochs))
            train_model(model, loss_function, optimizer,train_loader)
            test_model(model, loss_function, val_order)

def test_model(model, loss_function, data_loader):
    # Definindo o modelo para o modo de avaliacao
    model.eval()
    #variavel para armazer a perda
    current_loss = 0.0
    #variável para armazernar a precisão
    current_acc = 0
    # itera sobre o conjunto de validacao
    for i, (inputs, labels) in enumerate(data_loader):
        # Envia a entrada ou os rótulos para a GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        # passo adiante
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            #calcula as perdas na rede
            loss = loss_function(outputs, labels)
            # Estatisticas
            #perda nessa iteração
            current_loss += loss.item() * inputs.size(0)
            #precisão nessa iteração
            current_acc += torch.sum(predictions == labels.data)
            #perda total
            total_loss = current_loss / len(data_loader.dataset)
            #precisão total
            total_acc = current_acc.double() / len(data_loader.dataset)
            print('Test Loss: {:.4f}; Accuracy: {:.4f}'.format(total_loss, total_acc))
            
"""
Definir o segundo cenário para refinamento da rede original.
Similar ao cenário anterior, porém, toda a rede será treinada novamente
"""
def tl_fine_tuning(epochs=3):
    # Carregar o modelo pré-treinado
    model = models.resnet18(pretrained=True)
    # Substituir a última camada
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)
    # Transferir o modelo para a GPU
    model = model.to(device)
    # Função de Perda
    loss_function = nn.CrossEntropyLoss()
    # Otimização de todos os parâmetros
    optimizer = optim.Adam(model.parameters())
    # treinamento
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        train_model(model, loss_function, optimizer, train_loader)
        test_model(model, loss_function, val_order)
#invoca a função para rodar a rede de acordo com o primeiro cenário
tl_feature_extractor(epochs=5)
#invoca a função para rodar a rede de acordo com o segundo cenário
tl_fine_tuning(epochs=5)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth



Epoch 1/5
Train Loss: 0.0024; Accuracy:0.0001
Train Loss: 0.0049; Accuracy:0.0001
Train Loss: 0.0071; Accuracy:0.0003
Train Loss: 0.0095; Accuracy:0.0005
Train Loss: 0.0119; Accuracy:0.0006
Train Loss: 0.0142; Accuracy:0.0008
Train Loss: 0.0165; Accuracy:0.0010
Train Loss: 0.0187; Accuracy:0.0012
Train Loss: 0.0209; Accuracy:0.0015
Train Loss: 0.0231; Accuracy:0.0016
Train Loss: 0.0253; Accuracy:0.0019
Train Loss: 0.0275; Accuracy:0.0022
Train Loss: 0.0297; Accuracy:0.0024
Train Loss: 0.0318; Accuracy:0.0027
Train Loss: 0.0339; Accuracy:0.0029
Train Loss: 0.0361; Accuracy:0.0031
Train Loss: 0.0382; Accuracy:0.0033
Train Loss: 0.0400; Accuracy:0.0036
Train Loss: 0.0420; Accuracy:0.0040
Train Loss: 0.0441; Accuracy:0.0042
Train Loss: 0.0459; Accuracy:0.0046
Train Loss: 0.0478; Accuracy:0.0050
Train Loss: 0.0498; Accuracy:0.0053
Train Loss: 0.0517; Accuracy:0.0057
Train Loss: 0.0536; Accuracy:0.0061
Train Loss: 0.0556; Accuracy:0.0063
Train Loss: 0.0574; Accuracy:0.0067
Train Loss: 0.059